In [4]:
import numpy as np
import pandas as pd
import glob
import nltk 
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
import shutil
import random
from wordcloud import WordCloud
import gensim
import seaborn as sns
from textblob import TextBlob
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.datasets import load_files
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import FastText
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from scipy.sparse import hstack


nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ak2023\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ak2023\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ak2023\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

In [6]:
def remove_punctuation(text):
    # Define the pattern to match punctuation
    punctuation_pattern = r'[^\w\s]'
    # Replace punctuation with an empty string
    text_without_punctuation = re.sub(punctuation_pattern, '', text)
    # Normalize whitespace
    normalized_text = re.sub(r'\s+', ' ', text_without_punctuation)
    return normalized_text

# Text Processing
def preprocess_text(text, method='snowballstemming'):
    if not isinstance(text, str):
        return ''
    # Tokenize and lowercase
    tokens = word_tokenize(text.lower())

    # Define the set of stopwords
    stop_words = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
                "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he',
                'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its',
                'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
                'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 
                'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
                'but', 'if','or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
                'against', 'between', 'into', 'through','during', 'before', 'after', 'above', 'below', 'to', 'from',
                'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 
                'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both','each', 'few', 'more', 'most', 'other',
                'some', 'such','product', 'purchase', 'buy', 'bought', 'ordered', 'order', 'item', 'items', 
                'review', 'reviews', 'star', 'stars', 'rating', 'ratings', 'customer', 'customers', 
                'seller', 'sellers', 'seller', 'brand', 'brands', 'price', 'prices', 'money', 
                'value', 'quality', 'deal', 'deals','shipping', 'shipping', 'delivery', 'delivered', 'package', 
                'packaging','customer service', 'service', 'support', 'warranty', 'warranties', 
                'problem','use','one','work','machine','using','need','needed','cut','recieve',
                'glitter','pen','marker','folder','card', 'look','looking','color','background',
                'tool','design','made','craft','piece','store','size','markers','page','make','used' }


    tokens = [remove_punctuation(token) for token in tokens if token.isalnum() and token not in stop_words]

    if method == 'snowballstemming':
        stemmer = nltk.stem.SnowballStemmer('english')
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'porterstemming':
        stemmer = nltk.stem.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'lemmatization':
        lemmatizer = nltk.stem.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)
    

In [7]:
train_df['lemprocessing_text'] = train_df['Review'].apply(preprocess_text, method='lemmatization')

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

def early_stopping(monitor='val_loss', min_delta=0, patience=5, mode='auto'):
    return EarlyStopping(monitor=monitor, min_delta=min_delta, patience=patience, mode=mode)

# Function to create step decay learning rate scheduler
def step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5):
    def scheduler(epoch):
        return initial_lr * np.power(drop_factor, np.floor((1 + epoch) / epochs_drop))
    return scheduler

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from gensim.models import Word2Vec

# Split the data into features and target labels
X = train_df['lemprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)
# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100 


# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X, vector_size=embedding_size, window=5, min_count=1, workers=4)

# Convert words to Word2Vec embeddings
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embedding_size))
# Convert words to Word2Vec embeddings
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model with Word2Vec embeddings
def LSTM_model_with_Word2Vec_lem(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model_with_word2vec_lem = LSTM_model_with_Word2Vec_lem(max_len, vocab_size, embedding_size, embedding_matrix)

# Early stopping and decay callbacks
early_stop_callback = early_stopping(patience=5)
decay = LearningRateScheduler(step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5))
callbacks_list = [early_stop_callback, decay]

# Train the model
lstm_model_with_word2vec_lem.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_with_word2vec_lem.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)


In [ ]:
test_df['Review'] = test_df['Review'].fillna('')
X_test = test_df['Review'] 
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

# Predict ratings
predicted_labels = lstm_model_with_word2vec_lem.predict(X_test_padded)
predicted_ratings = predicted_labels.argmax(axis=1)

# Decode numerical labels back to original categories
predicted_sentiments = label_encoder.inverse_transform(predicted_ratings)

# Combine predictions with IDs
predictions_df = pd.DataFrame({'id': test_df['id'], 'overall': predicted_sentiments})

# Save predictions
predictions_df.to_csv('lstm_w2v_lem2.csv', index=False)

# Combine predictions with IDs
tm_df = pd.DataFrame({'id': test_df['id'], 'Review': test_df['Review'], 'overall': predicted_sentiments})

# Save predictions
tm_df.to_csv('lstm_w2v_lem2_tm.csv', index=False)